In [1]:
# Crea un netcdf con el viento a 925 hPa alineado con respecto
# a la fecha del onset de la temporada de lluvias.

import rioxarray

import pandas as pd
import numpy as np

import xarray as xr

# Datos básicos.

path_d = "../data/ERA5/"
path_r = "../results/onset/"

# Cantidad de días a promediar.
d = 20

# Variable, nivel de presión y región.
vars = [ "u", "v", "gp", "sst", "olr", "sp",
    "vidmf", "vivfu", "vivfv", "q" ]
levels = [ "925", "200", "700", "500" ]
region = [ "mexico" ]

# Años comprendidos en CHIRPS.
years = list( range(1981, 2021) )

# Cargamos la información de desfase para cada año.
shift = []
delta = []
shift.append ( pd.read_csv(path_r + "shift_CHIRPS.csv",
    index_col = "Año", squeeze = True) )
shift.append( pd.read_csv(path_r + "shift_withdrawal_CHIRPS.csv",
    index_col = "Año", squeeze = True) )
# Fecha pivote del desfase. 
for i in [0, 1]:
    delta.append( shift[i].max() )
    shift[i] = delta[i] - shift[i]

onset = [ "onset", "withdrawal" ]
anom = [ "", "_anom" ]

# Normal.
a = anom[0]

In [2]:
# viento 925 hPa.
n = 0

# Viento zonal y meridional.
u = []
for v in [0, 1]:
    fname = ( "onset_" + vars[v] + "_" + levels[n]
        + "_mean_"  + str(d) + "_dias" + a + ".grib" )
    u.append( xr.open_dataset( path_r + fname, engine = "cfgrib" ) )
    u[v] = u[v].drop_vars(["step", "isobaricInhPa", "valid_time"])

# Unimos en un solo netcdf.
ds = xr.merge( ( u[0], u[1] ) )

In [3]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_wind_"
    + levels[n] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [4]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_wind_"
    + levels[n] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [5]:
# viento 200 hPa.
n = 1

# Viento zonal y meridional.
u = []
for v in [0, 1]:
    fname = ( "onset_" + vars[v] + "_" + levels[n]
        + "_mean_"  + str(d) + "_dias" + a + ".grib" )
    u.append( xr.open_dataset( path_r + fname, engine = "cfgrib" ) )
    u[v] = u[v].drop_vars(["step", "isobaricInhPa", "valid_time"])

# Unimos en un solo netcdf.
ds = xr.merge( ( u[0], u[1] ) )

In [6]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_wind_"
    + levels[n] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [7]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_wind_"
    + levels[n] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [2]:
# viento 700 hPa.
n = 2

# Viento zonal y meridional.
u = []
for v in [0, 1]:
    fname = ( "onset_" + vars[v] + "_" + levels[n]
        + "_mean_"  + str(d) + "_dias" + a + ".grib" )
    u.append( xr.open_dataset( path_r + fname, engine = "cfgrib" ) )
    u[v] = u[v].drop_vars(["step", "isobaricInhPa", "valid_time"])

# Unimos en un solo netcdf.
ds = xr.merge( ( u[0], u[1] ) )

In [3]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_wind_"
    + levels[n] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [4]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_wind_"
    + levels[n] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [5]:
# viento 500 hPa.
n = 3

# Viento zonal y meridional.
u = []
for v in [0, 1]:
    fname = ( "onset_" + vars[v] + "_" + levels[n]
        + "_mean_"  + str(d) + "_dias" + a + ".grib" )
    u.append( xr.open_dataset( path_r + fname, engine = "cfgrib" ) )
    u[v] = u[v].drop_vars(["step", "isobaricInhPa", "valid_time"])

# Unimos en un solo netcdf.
ds = xr.merge( ( u[0], u[1] ) )

In [6]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_wind_"
    + levels[n] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [7]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_wind_"
    + levels[n] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [8]:
# Geopotencial a 925 hPa.
n = 0
v = 2

fname = ( "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".grib" )
ds = xr.open_dataset( path_r + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])

In [9]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
    + "_" + levels[n] + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [10]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
    + "_" + levels[n] + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [11]:
# Geopotencial a 200 hPa.
n = 1
v = 2

fname = ( "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".grib" )
ds = xr.open_dataset( path_r + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])

In [12]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
    + "_" + levels[n] + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [13]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
    + "_" + levels[n] + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [15]:
# Humedad específica a 925 hPa.
n = 0
v = 9

fname = ( "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".grib" )
ds = xr.open_dataset( path_r + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])

In [16]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
    + "_" + levels[n] + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [17]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
    + "_" + levels[n] + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [18]:
# Humedad específica a 200 hPa.
n = 1
v = 9

fname = ( "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".grib" )
ds = xr.open_dataset( path_r + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])

In [19]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
    + "_" + levels[n] + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [20]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
    + "_" + levels[n] + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [21]:
# Humedad específica a 700 hPa.
n = 2
v = 9

fname = ( "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".grib" )
ds = xr.open_dataset( path_r + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
    + "_" + levels[n] + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
    + "_" + levels[n] + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Humedad específica a 500 hPa.
n = 3
v = 9

fname = ( "onset_" + vars[v] + "_" + levels[n] + "_mean_" 
    + str(d) + "_dias" + a + ".grib" )
ds = xr.open_dataset( path_r + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])

In [ ]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
    + "_" + levels[n] + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
    + "_" + levels[n] + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [14]:
# Temperatura superficial del océano.
v = 3

# Normal.
fname = ( "onset_" + vars[v] + "_mean_" + str(d) + "_dias" + a + ".grib" )
ds = xr.open_dataset( path_r + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "surface", "valid_time"])

In [15]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_"
    + vars[v] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [16]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_"
    + vars[v] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [17]:
# OLR
v = 4

# Normal.
fname = ( "onset_" + vars[v] + "_mean_" + str(d) + "_dias" + a + ".grib" )
ds = xr.open_dataset( path_r + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "surface", "valid_time"])

In [18]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_"
    + vars[v] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [19]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_"
    + vars[v] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [20]:
# Presión superficial
v = 5

# Normal.
fname = ( "onset_" + vars[v] + "_mean_" + str(d) + "_dias" + a + ".grib" )
ds = xr.open_dataset( path_r + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "surface", "valid_time"])

In [21]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_"
    + vars[v] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [22]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_"
    + vars[v] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [25]:
# Vertical integral moisture divergence
v = 6

# Normal.
fname = ( "onset_" + vars[v] + "_mean_" + str(d) + "_dias" + a + ".grib" )
ds = xr.open_dataset( path_r + fname, engine = "cfgrib" )
ds = ds.drop_vars(["step", "entireAtmosphere", "valid_time"])

In [26]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_"
    + vars[v] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [27]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_"
    + vars[v] + "_mean_"  + str(d) + "_dias" + a + ".nc" )

In [28]:
# Vertical integrated moisture flow

# Viento zonal y meridional.
u = []
for v in [0, 1]:
    fname = ( "onset_" + vars[v + 7] + "_mean_" 
        + str(d) + "_dias" + a + ".grib" )
    u.append( xr.open_dataset( path_r + fname, engine = "cfgrib" ) )
    u[v] = u[v].drop_vars(["step", "entireAtmosphere", "valid_time"])

# Unimos en un solo netcdf.
ds = xr.merge( ( u[0], u[1] ) )

v = 7

In [29]:
# Onset, composite de la distribución espacial.
j = 0
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v][:-1]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )

In [ ]:
# Withdrawal, composite de la distribución espacial.
j = 1
# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos las fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift[j].iloc[i] } )
# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Agregamos los primeros días del año después del último día del año.
x = composite.isel(dayofyear = slice(0, 60))
x["dayofyear"] = np.arange(366, 366 + 60)
composite = xr.concat( (composite, x), dim = "dayofyear" )
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice(
    delta[j] - 60, delta[j] + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)
# Guardamos el netcdf.
composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v][:-1]
    + "_mean_" + str(d) + "_dias" + a + ".nc" )